## Imports

In [5]:
import pickle
import os
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import sys
sys.path.insert(0, os.path.abspath('../..'))
from src import references as refs

In [7]:
# Load
corpus_dir = '/Users/pkalluri/projects/clarity/bert-vis/big-data/wiki-large/wiki-split/'

corpus_contexts = pickle.load(open(os.path.join(corpus_dir, refs.contexts_fn), 'rb'))
n_corpus_contexts = len(corpus_contexts)
corpus_acts = np.load(os.path.join(corpus_dir, refs.acts_fn))
layers = corpus_acts.files

In [8]:
# Map toks to ids
id_to_tok = {}
tok_to_id = {}
corpus_toks = []
corpus_toks_ids = []
curr_id = 0
for context in corpus_contexts:
    doc, pos = context
    tok = doc[pos]
    if not tok in tok_to_id:
        id_to_tok[curr_id] = tok
        tok_to_id[tok] = curr_id
        curr_id += 1
    corpus_toks.append(tok)
    corpus_toks_ids.append(tok_to_id[tok])


In [9]:
len(id_to_tok)

23051

In [10]:
n_corpus_contexts * .009

5276.241

In [22]:
model = LogisticRegression(max_iter=1000).fit(np.eye(3), list(range(3)))
model.predict([[0,0,1], [0,1,1]])

array([2, 1])

In [23]:
# Probe to what extent embeddings contain info about tok ids

# prep target data (the tokens' idxs)
train_fraction, test_fraction = .009, .001
n_train = int(train_fraction*n_corpus_contexts)
train_toks_ids = corpus_toks_ids[:n_train]
test_toks_ids = corpus_toks_ids[n_train:]

scores = {}  # each layer get a score for to what extent it contains the info
scaler = StandardScaler()
for layer in layers:
    print(layer)
    # prep input data (the tokens' acts)
    _corpus_acts_scaled = scaler.fit_transform(corpus_acts[layer])
    print('scaled')
    train_acts = _corpus_acts_scaled[:n_train]
    test_acts = _corpus_acts_scaled[n_train:]
    # probe
    print('fit')
    model = LogisticRegression(max_iter=1000).fit(train_acts, train_toks_ids)
    print('eval')
    train_score = model.score(train_acts, train_toks_ids)
    for i in range(3):
        print(id_to_tok[model.predict([train_acts[i]], [train_toks_ids[i]])[0]], corpus_context[i])
    test_score = model.score(test_acts, test_toks_ids)
    del model
    print(train_score, test_score)
    scores[layer] = (train_score, test_score)


arr_0


KeyboardInterrupt: 

In [ ]:
# Show results
for layer in layers:
    print(np.average(accuracies[layer]))